**Instalación de las librerias a utilizar**


In [ ]:
# !pip install pandas
# !pip install tqdm
# !pip install datetime
# !pip install numpy
# !pip install gspread oauth2client pandas
# !pip install gspread-dataframe
# !pip install --upgrade google-api-python-client
# !pip install openpyxl
# !pip install gdown



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\cgrub\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/186.0 kB ? eta -:--:--
   ---------------------------------------  184.3/186.0 kB 5.4 MB/s eta 0:00:01
   ---------------------------------------- 186.0/186.0 kB 3.7 MB/s eta 0:00:00


In [8]:
import pandas as pd
import datetime
from tqdm import tqdm
import re
import unicodedata

*Importar los datos directamente de la página de datos abiertos, esto puede tardar varias horas, ya que son más de 4 millones de datos*
### Link de los datos filtrados del 2023 al 2025:
 https://www.datos.gov.co/Gastos-Gubernamentales/SECOP-II-Contratos-Electr-nicos/jbjy-vk9h/explore/query/SELECT%0A%20%20%60nombre_entidad%60%2C%0A%20%20%60nit_entidad%60%2C%0A%20%20%60departamento%60%2C%0A%20%20%60ciudad%60%2C%0A%20%20%60localizaci_n%60%2C%0A%20%20%60sector%60%2C%0A%20%20%60proceso_de_compra%60%2C%0A%20%20%60id_contrato%60%2C%0A%20%20%60referencia_del_contrato%60%2C%0A%20%20%60estado_contrato%60%2C%0A%20%20%60descripcion_del_proceso%60%2C%0A%20%20%60tipo_de_contrato%60%2C%0A%20%20%60modalidad_de_contratacion%60%2C%0A%20%20%60justificacion_modalidad_de%60%2C%0A%20%20%60fecha_de_firma%60%2C%0A%20%20%60fecha_de_inicio_del_contrato%60%2C%0A%20%20%60fecha_de_fin_del_contrato%60%2C%0A%20%20%60fecha_de_inicio_de_ejecucion%60%2C%0A%20%20%60fecha_de_fin_de_ejecucion%60%2C%0A%20%20%60condiciones_de_entrega%60%2C%0A%20%20%60tipodocproveedor%60%2C%0A%20%20%60documento_proveedor%60%2C%0A%20%20%60proveedor_adjudicado%60%2C%0A%20%20%60liquidaci_n%60%2C%0A%20%20%60reversion%60%2C%0A%20%20%60valor_del_contrato%60%2C%0A%20%20%60valor_facturado%60%2C%0A%20%20%60valor_pendiente_de_pago%60%2C%0A%20%20%60valor_pagado%60%2C%0A%20%20%60valor_pendiente_de_ejecucion%60%2C%0A%20%20%60dias_adicionados%60%2C%0A%20%20%60urlproceso%60%2C%0A%20%20%60nombre_representante_legal%60%2C%0A%20%20%60nacionalidad_representante_legal%60%2C%0A%20%20%60ultima_actualizacion%60%2C%0A%20%20%60codigo_entidad%60%2C%0A%20%20%60codigo_proveedor%60%2C%0A%20%20%60fecha_inicio_liquidacion%60%2C%0A%20%20%60fecha_fin_liquidacion%60%2C%0A%20%20%60objeto_del_contrato%60%2C%0A%20%20%60duraci_n_del_contrato%60%0AWHERE%0A%20%20%60fecha_de_firma%60%0A%20%20%20%20BETWEEN%20%222023-01-01T09%3A20%3A50%22%20%3A%3A%20floating_timestamp%0A%20%20%20%20AND%20%222025-12-31T09%3A20%3A50%22%20%3A%3A%20floating_timestamp/page/filter

In [9]:
# URL base del dataset en Datos Abiertos
BASE_URL = "https://www.datos.gov.co/resource/qmzu-gj57.csv"

# Obtener el total de filas disponibles antes de la descarga
df_info = pd.read_csv(BASE_URL, nrows=1)  # Intentamos leer solo una fila para que sea rápido
total_rows_estimated = df_info.shape[0]  # Obtener la cantidad de filas

print(f"Se estima que hay un total de {total_rows_estimated} filas en el dataset.")

# Parámetros para paginar (bloques de 1000 filas)
LIMIT = 1000
OFFSET = 0

# Lista para almacenar los datos
data_list = []

# Contador de filas importadas
total_rows = 0

print("Descargando datos...")
with tqdm(unit=" filas") as pbar:  # Inicializa la barra de progreso
    while True:
        # Construir URL con paginación
        url = f"{BASE_URL}?$limit={LIMIT}&$offset={OFFSET}"
        
        # Cargar los datos en un DataFrame temporal
        df_temp = pd.read_csv(url)
        
        # Si no hay más datos, detener la iteración
        if df_temp.empty:
            break
        
        # Agregar datos a la lista
        data_list.append(df_temp)
        
        # Actualizar contador y barra de progreso
        total_rows += len(df_temp)
        pbar.update(len(df_temp))  # Actualizar barra con nuevas filas
        
        # Aumentar el OFFSET para la siguiente página
        OFFSET += LIMIT

# Concatenar todos los bloques en un único DataFrame
df_proveedores = pd.concat(data_list, ignore_index=True)

# Mostrar las primeras filas
print(df_proveedores.head())

# Opcional: Guardar en un archivo CSV
df_proveedores.to_csv("datos_abiertos.csv", index=False)

print(f"\n📥 Descarga completada. Total de filas importadas: {total_rows}")


Se estima que hay un total de 1 filas en el dataset.
Descargando datos...


1345430 filas [38:38, 580.41 filas/s]


      codigo                          nombre         nit es_entidad es_grupo  \
0  728143553           MICHEL DAVID INSUASTY  1122785124         No       No   
1  728173774         NILSON CARVAJAL MERCADO    12596743         No       No   
2  717500045    CONSTRUCTORA PROVIDENCIA SAS   901508733         No       No   
3  703168765              SERVIMOS SGC S.A.S   823003799         No       No   
4  718630643  MARIA ISABEL SEPULVEDA ZULUAGA  1107520234         No       No   

  esta_activa           fecha_creacion codigo_categoria_principal  \
0          Si  2024-08-28T00:00:00.000                No Definido   
1          Si  2024-08-30T00:00:00.000                No Definido   
2          Si  2022-03-31T00:00:00.000                No Definido   
3          Si  2017-11-22T00:00:00.000                No Definido   
4          Si  2022-08-01T00:00:00.000                No Definido   

  descripcion_categoria_principal     telefono  ...  municipio    sitio_web  \
0                     No 

In [10]:
df_proveedores

,codigo,nombre,nit,es_entidad,es_grupo,esta_activa,fecha_creacion,codigo_categoria_principal,descripcion_categoria_principal,telefono,...,municipio,sitio_web,tipo_empresa,nombre_representante_legal,tipo_doc_representante_legal,n_mero_doc_representante_legal,telefono_representante_legal,correo_representante_legal,espyme,ubicacion
0,728143553,MICHEL DAVID INSUASTY,1122785124,No,No,Si,2024-08-28T00:00:00.000,No Definido,No Definido,3147024045,...,SIBUNDOY,No Provisto,PERSONA NATURAL COLOMBIANA,MICHEL DAVID INSUASTY LUNA,Cédula de Ciudadanía,1122785124,3147024045,INSUASTY_@HOTMAIL.COM,NO,CO-PUT-86749
1,728173774,NILSON CARVAJAL MERCADO,12596743,No,No,Si,2024-08-30T00:00:00.000,No Definido,No Definido,3014560028,...,PLATO,No Provisto,OTRO,NILSON ENRIQUE CARVAJAL MERCADO,Cédula de Ciudadanía,12596743,3014560028,NCARVAJALMERCADO@GMAIL.COM,NO,CO-MAG-47555
2,717500045,CONSTRUCTORA PROVIDENCIA SAS,901508733,No,No,Si,2022-03-31T00:00:00.000,No Definido,No Definido,310 2475018,...,ARAUCA,No Provisto,SOCIEDAD POR ACCIONES SIMPLIFICADA,SANDRA AMAYA,NIT,901508733,3167416603,constructoraprovidenciasas@gmail.com,SI,CO-ARA-81001
3,703168765,SERVIMOS SGC S.A.S,823003799,No,No,Si,2017-11-22T00:00:00.000,No Definido,No Definido,3113585798,...,SINCELEJO,No Provisto,SOCIEDAD POR ACCIONES SIMPLIFICADA,PIEDAD ROCIO CONTRERAS ALMANZA,NIT,823003799,3162329257,y.solano12@yahoo.com,SI,CO-SUC-70001
4,718630643,MARIA ISABEL SEPULVEDA ZULUAGA,1107520234,No,No,Si,2022-08-01T00:00:00.000,No Definido,No Definido,3172199665,...,CALI,No Provisto,PERSONA NATURAL COLOMBIANA,MARIA ISABEL SEPULVEDA ZULUAGA,Cédula de Ciudadanía,1107520234,3174540741,sepul212020@hotmail.com,NO,CO-VAC-76001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345425,714529385,JENNY ALEJANDRA FANDIÑO CARRILLO,1016077288,No,No,Si,2021-07-27T00:00:00.000,No Definido,No Definido,3134729249,...,BOGOTa,No Provisto,PERSONA NATURAL COLOMBIANA,Jenny Alejandra Fandiño,Cédula de Ciudadanía,1016077288,3012661059,alefandinocd86@gmail.com,NO,CO-DC-11001
1345426,730115896,ANDRES FELIPE PLAZAS LOPEZ,1010064659,No,No,Si,2025-01-24T00:00:00.000,No Definido,No Definido,3213508049,...,CHiA,No Provisto,PERSONA NATURAL COLOMBIANA,ANDRES FELIPE PLAZAS LOPEZ,Cédula de Ciudadanía,1010064659,3213508049,plazaslopez96@gmail.com,NO,CO-CUN-25175
1345427,730105962,KATHERINE PULGARIN MARTINEZ,1097400920,No,No,Si,2025-01-23T00:00:00.000,No Definido,No Definido,31283309178,...,ARMENIA,No Provisto,OTRO,Katherine Pulgarin martinez,Cédula de Ciudadanía,1097400920,3128309178,Kathaa.18pm@gmail.com,NO,CO-QUI-63001
1345428,730112554,KAREN TATIANA SUEVIS GOMEZ,1065836860,No,No,Si,2025-01-23T00:00:00.000,No Definido,No Definido,3008798671,...,BOGOTa,No Provisto,PERSONA NATURAL COLOMBIANA,Karen Tatiana Suevis Gomez,Cédula de Ciudadanía,1065836860,3008798671,abog.karensuevisg@gmail.com,NO,CO-DC-11001


In [11]:
# Ruta del archivo CSV en tu sistema
csv_path = r"C:\Users\cgrub\Contratos_Secop_Col\Contratos_SECOP_Col\SECOP CONTRATOS.csv"

# Cargar el archivo CSV en un DataFrame
try:
    df = pd.read_csv(csv_path, encoding="utf-8")  # Usa encoding="latin1" si hay problemas con caracteres especiales
    print("Archivo cargado con éxito.")
    print(f"Total de filas: {df.shape[0]}")
    print(df.head())  # Mostrar las primeras filas
except FileNotFoundError:
    print("❌ Error: No se encontró el archivo en la ruta especificada.")
except Exception as e:
    print(f"⚠️ Ocurrió un error: {e}")

C:\Users\cgrub\AppData\Local\Temp\ipykernel_4328\543116558.py:6: DtypeWarning: Columns (17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path, encoding="utf-8")  # Usa encoding="latin1" si hay problemas con caracteres especiales


Archivo cargado con éxito.
Total de filas: 2069859
                                      Nombre Entidad  Nit Entidad  \
0  E.S.E HOSPITAL NUESTRA SEÑORA DEL CARMEN DEL C...    860020094   
1  EMPRESA SOCIAL DEL ESTADO DEL MUNICIPIO DE VIL...    822002459   
2                          Municipio de Montenegro--    890000858   
3       HOSPITAL REINA SOFIA DE ESPAÑA LERIDA TOLIMA    890706823   
4      EMPRESA SOCIAL DEL ESTADO SANTIAGO DE TUNJA *   8200038502   

   Departamento         Ciudad                           Localización  \
0  Cundinamarca     El Colegio  Colombia,  Cundinamarca ,  El Colegio   
1          Meta  Villavicencio       Colombia,  Meta ,  Villavicencio   
2       Quindío     Montenegro       Colombia,  Quindío ,  Montenegro   
3        Tolima         Lérida            Colombia,  Tolima ,  Lérida   
4        Boyacá          Tunja             Colombia,  Boyacá ,  Tunja   

                      Sector Proceso de Compra         ID Contrato  \
0  Salud y Protección Soc

In [30]:
# Contar filas antes
filas_antes = len(df)

# Filtrar el DataFrame
df = df[df['Tipo de Contrato'] != 'Prestación de servicios']

# Contar filas después
filas_despues = len(df)

# Mostrar resultados
print(f"Filas antes: {filas_antes}")
print(f"Filas después: {filas_despues}")
print(f"Filas eliminadas: {filas_antes - filas_despues}")


Filas antes: 2069859
Filas después: 272033
Filas eliminadas: 1797826


In [31]:
# Mostrar información general del DataFrame de SECOP
print(df.dtypes)
print(df.columns)

Nombre Entidad                                     object
Nit Entidad                                         int64
Departamento                                       object
Ciudad                                             object
Localización                                       object
Sector                                             object
Proceso de Compra                                  object
ID Contrato                                        object
Referencia del Contrato                            object
Estado Contrato                                    object
Descripcion del Proceso                            object
Tipo de Contrato                                   object
Modalidad de Contratacion                          object
Justificacion Modalidad de Contratacion            object
Fecha de Firma                             datetime64[ns]
Fecha de Inicio del Contrato               datetime64[ns]
Fecha de Fin del Contrato                  datetime64[ns]
Fecha de Inici

*Limpieda de datos*  
* Validación de datos tipo fecha  
* Validación de datos tipo int o float  
* Convertir "Duración del contrato" a días  

In [32]:
# Convertir múltiples columnas a tipo datetime
columnas_fecha = [
    "Fecha de Firma", "Fecha de Inicio del Contrato", "Fecha de Fin del Contrato",
    "Fecha de Inicio de Ejecucion", "Fecha de Fin de Ejecucion", "Ultima Actualizacion",
    "Fecha Inicio Liquidacion", "Fecha Fin Liquidacion"
]

df[columnas_fecha] = df[columnas_fecha].apply(pd.to_datetime, errors="coerce")

# Ver los tipos de datos
print(df.dtypes)


df

C:\Users\cgrub\AppData\Local\Temp\ipykernel_4328\581524281.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[columnas_fecha] = df[columnas_fecha].apply(pd.to_datetime, errors="coerce")


Nombre Entidad                                     object
Nit Entidad                                         int64
Departamento                                       object
Ciudad                                             object
Localización                                       object
Sector                                             object
Proceso de Compra                                  object
ID Contrato                                        object
Referencia del Contrato                            object
Estado Contrato                                    object
Descripcion del Proceso                            object
Tipo de Contrato                                   object
Modalidad de Contratacion                          object
Justificacion Modalidad de Contratacion            object
Fecha de Firma                             datetime64[ns]
Fecha de Inicio del Contrato               datetime64[ns]
Fecha de Fin del Contrato                  datetime64[ns]
Fecha de Inici

,Nombre Entidad,Nit Entidad,Departamento,Ciudad,Localización,Sector,Proceso de Compra,ID Contrato,Referencia del Contrato,Estado Contrato,...,Nombre Representante Legal,Nacionalidad Representante Legal,Ultima Actualizacion,Codigo Entidad,Codigo Proveedor,Fecha Inicio Liquidacion,Fecha Fin Liquidacion,Objeto del Contrato,Duración del contrato,Duración en Días
1,EMPRESA SOCIAL DEL ESTADO DEL MUNICIPIO DE VIL...,822002459,Meta,Villavicencio,"Colombia, Meta , Villavicencio",Salud y Protección Social,CO1.BDOS.3677639,CO1.PCCNTR.4364202,039-2023,terminado,...,EVELYN ROSA CALLE DE LOS REYES,CO,2024-10-31,702575192,720368331,NaT,NaT,PRESTACION DE SERVICIOS PROFESIONALES COMO BAC...,270 Dia(s),270.0
6,CENTRO DE DIAGNOSTICO AUTOMOTOR DE NARIÑO LTDA,800206629,Nariño,Pasto,"Colombia, Nariño , Pasto",Transporte,CO1.BDOS.3692626,CO1.PCCNTR.4365365,PS2023-002,En ejecución,...,GERMAN MONTENEGRO FAJARDO,CO,NaT,708210232,701215097,NaT,NaT,Prestar los servicios profesionales de Revisor...,90 Dia(s),90.0
9,RNEC,899999040,Distrito Capital de Bogotá,Bogotá,"Colombia, Bogotá, Bogotá",No aplica/No pertenece,CO1.BDOS.3691081,CO1.PCCNTR.4364799,DS-001-AGUADA-2023-RNEC-SANTANDER,Cerrado,...,ZONIA ALEXANDRA BENAVIDES TRIANA,CO,2024-03-19,704868611,711614644,NaT,NaT,El ARRENDADOR se compromete a dar en arrendami...,13 Mes(es),390.0
15,ALCALDIA LA LLANADA,8001498940,Nariño,La Llanada,"Colombia, Nariño , La Llanada",No aplica/No pertenece,CO1.BDOS.3667289,CO1.PCCNTR.4358427,MC 083 2022,Cerrado,...,IVAN DAVID AYALA ORTEGA,CO,NaT,709823447,719218182,NaT,NaT,SUMINISTRO DE ELEMENTOS DE PAPELERÍA Y MATERIA...,1 Dia(s),1.0
25,EMPRESA SOCIAL DEL ESTADO E.S.E. HOSPITAL REGI...,807008857,Norte de Santander,Tibú,"Colombia, Norte de Santander , Tibú",Salud y Protección Social,CO1.BDOS.3680738,CO1.PCCNTR.4358085,088-2023,En ejecución,...,WILSON ALBEERTO CORONADO GONZALEZ,CO,NaT,713471803,711374082,NaT,NaT,PRESTACION DE SERVICIOS EN APOYO A LA OFICINA ...,4 Mes(es),120.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2069819,ICBF REGIONAL BOLIVAR,899999239,Bolívar,Cartagena,"Colombia, Bolívar , Cartagena",Salud y Protección Social,CO1.BDOS.7686721,CO1.PCCNTR.7549002,13004782025,Activo,...,FUNDESINC FUNDESINC,CO,NaT,704148527,729311233,2025-10-01,2028-03-30,Complementar la atención de niños y niñas a tr...,8 Mes(es),240.0
2069827,ICBF REGIONAL QUINDIO,899999239,Antioquia,Armenia,"Colombia, Antioquia , Armenia",Salud y Protección Social,CO1.BDOS.7685935,CO1.PCCNTR.7548611,63001512025,Activo,...,Cooperativa Coohobienestar,CO,NaT,704193143,707715488,NaT,NaT,COMPLEMENTAR LA ATENCIÓN DE NIÑOS Y NIÑAS A TR...,215 Dia(s),215.0
2069830,ICBF REGIONAL QUINDIO,899999239,Antioquia,Armenia,"Colombia, Antioquia , Armenia",Salud y Protección Social,CO1.BDOS.7685352,CO1.PCCNTR.7548317,63001492025,Activo,...,CLAUDIA LILIANA CASTIBLANCO RIOS,CO,NaT,704193143,709330146,NaT,NaT,COMPLEMENTAR LA ATENCIÓN DE NIÑOS A TRAVÉS DE ...,215 Dia(s),215.0
2069836,ICBF REGIONAL QUINDIO,899999239,Antioquia,Armenia,"Colombia, Antioquia , Armenia",Salud y Protección Social,CO1.BDOS.7686218,CO1.PCCNTR.7548711,63001522025,Activo,...,Guillermo José Arcila soto,CO,NaT,704193143,702380056,NaT,NaT,COMPLEMENTAR LA ATENCIÓN DE NIÑOS Y NIÑAS A TR...,215 Dia(s),215.0


In [33]:
# Función para eliminar tildes y normalizar texto
def normalizar_texto(texto):
    return ''.join(
        c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn'
    )

# Función para convertir meses y días a días
def convertir_a_dias(valor):
    if pd.isna(valor) or not isinstance(valor, str):  # Manejar valores nulos o no strings
        return None

    valor = valor.strip()  # Eliminar espacios extra
    valor = normalizar_texto(valor)  # Eliminar tildes para evitar problemas
    numero = re.findall(r'\d+', valor)  # Extraer solo los números

    if not numero:  # Si no hay número, retornar None
        return None

    numero = int(numero[0])  # Convertir el primer número encontrado a entero

    if "Mes" in valor:  # Convertir meses a días
        return numero * 30
    elif "Dia" in valor:  # Manejar días correctamente
        return numero
    else:
        return None  # Para valores inesperados

# Aplicar la conversión al DataFrame existente
df["Duración en Días"] = df["Duración del contrato"].astype(str).str.strip().apply(convertir_a_dias)

# Verificar resultados
print(df[["Duración del contrato", "Duración en Días"]].head())


   Duración del contrato  Duración en Días
1             270 Dia(s)             270.0
6              90 Dia(s)              90.0
9             13 Mes(es)             390.0
15              1 Dia(s)               1.0
25             4 Mes(es)             120.0


C:\Users\cgrub\AppData\Local\Temp\ipykernel_4328\1877599991.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Duración en Días"] = df["Duración del contrato"].astype(str).str.strip().apply(convertir_a_dias)


In [34]:
# Mostrar información general del DataFrame de PROVEEDORES
print(df_proveedores.dtypes)
print(df_proveedores.columns)

codigo                           Int64
nombre                          object
telefono                        object
fax                             object
correo                          object
direccion                       object
pais                            object
departamento                    object
municipio                       object
telefono_representante_legal    object
correo_representante_legal      object
ubicacion                       object
dtype: object
Index(['codigo', 'nombre', 'telefono', 'fax', 'correo', 'direccion', 'pais',
       'departamento', 'municipio', 'telefono_representante_legal',
       'correo_representante_legal', 'ubicacion'],
      dtype='object')


Limpieza y Transformación de Datos en `df_proveedores`

Este script realiza la conversión y selección de datos en el DataFrame `df_proveedores` para facilitar su análisis y uso posterior.

1. Conversión del Código a Entero  
La columna `codigo` originalmente está en formato `object` (cadena de texto). Para asegurar su correcto tratamiento numérico, se convierte a `Int64`, manejando posibles errores mediante `errors="coerce"`, lo que convierte valores no numéricos en `NaN`.


In [35]:
# Convertir 'codigo' a entero, manejando posibles errores
df_proveedores["codigo"] = pd.to_numeric(df_proveedores["codigo"], errors="coerce").astype("Int64")

# Seleccionar solo las columnas requeridas
columnas_requeridas = [
    "codigo", "nombre", "telefono", "fax", "correo", "direccion", 
    "pais", "departamento", "municipio", "telefono_representante_legal", 
    "correo_representante_legal", "ubicacion"
]

df_proveedores = df_proveedores[columnas_requeridas]

# Verificar cambios
print(df_proveedores.info())
df_proveedores.head()


C:\Users\cgrub\AppData\Local\Temp\ipykernel_4328\1083030422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_proveedores["codigo"] = pd.to_numeric(df_proveedores["codigo"], errors="coerce").astype("Int64")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1345430 entries, 0 to 1345429
Data columns (total 12 columns):
 #   Column                        Non-Null Count    Dtype 
---  ------                        --------------    ----- 
 0   codigo                        1345405 non-null  Int64 
 1   nombre                        1345389 non-null  object
 2   telefono                      1345430 non-null  object
 3   fax                           1345267 non-null  object
 4   correo                        1345430 non-null  object
 5   direccion                     1345368 non-null  object
 6   pais                          1345430 non-null  object
 7   departamento                  1345430 non-null  object
 8   municipio                     1345430 non-null  object
 9   telefono_representante_legal  1345430 non-null  object
 10  correo_representante_legal    1345430 non-null  object
 11  ubicacion                     1345430 non-null  object
dtypes: Int64(1), object(11)
memory usage: 124.

,codigo,nombre,telefono,fax,correo,direccion,pais,departamento,municipio,telefono_representante_legal,correo_representante_legal,ubicacion
0,728143553,MICHEL DAVID INSUASTY,3147024045,No Provisto,INSUASTY_@HOTMAIL.COM,CARRERA 10 CALLE 20,CO,PUTUMAYO,SIBUNDOY,3147024045,INSUASTY_@HOTMAIL.COM,CO-PUT-86749
1,728173774,NILSON CARVAJAL MERCADO,3014560028,No Provisto,NCARVAJALMERCADO@GMAIL.COM,CALLE 6 # 15A-35,CO,MAGDALENA,PLATO,3014560028,NCARVAJALMERCADO@GMAIL.COM,CO-MAG-47555
2,717500045,CONSTRUCTORA PROVIDENCIA SAS,310 2475018,No Provisto,CONSTRUCTORAPROVIDENCIASAS@GMAIL.COM,CRA 16 20-12,CO,ARAUCA,ARAUCA,3167416603,constructoraprovidenciasas@gmail.com,CO-ARA-81001
3,703168765,SERVIMOS SGC S.A.S,3113585798,No Provisto,GERENCIASERVIMOS@GMAIL.COM,TRANVERSAL 31 NRO. 33 - 60,CO,SUCRE,SINCELEJO,3162329257,y.solano12@yahoo.com,CO-SUC-70001
4,718630643,MARIA ISABEL SEPULVEDA ZULUAGA,3172199665,No Provisto,SEPUL212020@HOTMAIL.COM,CARRERA 112 # 44-21,CO,VALLE DEL CAUCA,CALI,3174540741,sepul212020@hotmail.com,CO-VAC-76001


In [38]:
print(df_proveedores["codigo"].nunique())  # Cantidad de valores únicos en 'codigo'
print(df["Codigo Proveedor"].nunique())  # Cantidad de valores únicos en 'Codigo Proveedor'


1012386
81762


In [39]:
codigos_df = set(df["Codigo Proveedor"].unique())
codigos_proveedores = set(df_proveedores["codigo"].unique())

codigos_extra_en_proveedores = codigos_proveedores - codigos_df
codigos_faltantes_en_proveedores = codigos_df - codigos_proveedores

print(f"Códigos en df_proveedores que NO están en df: {len(codigos_extra_en_proveedores)}")
print(f"Códigos en df que NO están en df_proveedores: {len(codigos_faltantes_en_proveedores)}")


Códigos en df_proveedores que NO están en df: 959808
Códigos en df que NO están en df_proveedores: 29183


In [56]:
# Asegurar que los tipos de datos sean iguales
df["Codigo Proveedor"] = pd.to_numeric(df["Codigo Proveedor"], errors="coerce").astype("Int64")
df_proveedores["codigo"] = pd.to_numeric(df_proveedores["codigo"], errors="coerce").astype("Int64")

# Unir los DataFrames con merge (similar a un VLOOKUP)
df_final = df.merge(df_proveedores, how="left", left_on="Codigo Proveedor", right_on="codigo")

# Eliminar la columna duplicada 'codigo' si ya no es necesaria y las que no son necesarias
df_final.drop(columns=["Codigo Proveedor", "Ciudad", "Departamento"], inplace=True)


# Verificar la unión
print(df_final.info())  # Revisión de la estructura
df_final

C:\Users\cgrub\AppData\Local\Temp\ipykernel_4328\1916449497.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Codigo Proveedor"] = pd.to_numeric(df["Codigo Proveedor"], errors="coerce").astype("Int64")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330331 entries, 0 to 330330
Data columns (total 51 columns):
 #   Column                                   Non-Null Count   Dtype         
---  ------                                   --------------   -----         
 0   Nombre Entidad                           330331 non-null  object        
 1   Nit Entidad                              330331 non-null  int64         
 2   Localización                             330331 non-null  object        
 3   Sector                                   330331 non-null  object        
 4   Proceso de Compra                        330331 non-null  object        
 5   ID Contrato                              330331 non-null  object        
 6   Referencia del Contrato                  330331 non-null  object        
 7   Estado Contrato                          330313 non-null  object        
 8   Descripcion del Proceso                  330331 non-null  object        
 9   Tipo de Contrato          

,Nombre Entidad,Nit Entidad,Localización,Sector,Proceso de Compra,ID Contrato,Referencia del Contrato,Estado Contrato,Descripcion del Proceso,Tipo de Contrato,...,telefono,fax,correo,direccion,pais,departamento,municipio,telefono_representante_legal,correo_representante_legal,ubicacion
0,EMPRESA SOCIAL DEL ESTADO DEL MUNICIPIO DE VIL...,822002459,"Colombia, Meta , Villavicencio",Salud y Protección Social,CO1.BDOS.3677639,CO1.PCCNTR.4364202,039-2023,terminado,PRESTACION DE SERVICIOS PROFESIONALES COMO BAC...,Decreto 092 de 2017,...,3228982864,No Provisto,DANAVHC1003@GMAIL.COM,CALLE 39 C N 7-22 MANANTIAL,CO,META,VILLAVICENCIO,3228982864,danavhc1003@gmail.com,CO-MET-50001
1,CENTRO DE DIAGNOSTICO AUTOMOTOR DE NARIÑO LTDA,800206629,"Colombia, Nariño , Pasto",Transporte,CO1.BDOS.3692626,CO1.PCCNTR.4365365,PS2023-002,En ejecución,Prestar los servicios profesionales de Revisor...,Decreto 092 de 2017,...,7315832,7315832,HWILSONH11@GMAIL.COM,CALLE 19 NO. 28 - 72 OFICINA 302,CO,NARIÑO,PASTO,3187767039,codigosdiangmf@gmail.com,CO-NAR-52001
2,RNEC,899999040,"Colombia, Bogotá, Bogotá",No aplica/No pertenece,CO1.BDOS.3691081,CO1.PCCNTR.4364799,DS-001-AGUADA-2023-RNEC-SANTANDER,Cerrado,El ARRENDADOR se compromete a dar en arrendami...,Arrendamiento de inmuebles,...,3212087249,No Provisto,MARIPOSAAZUL.77@OUTLOOK.ES,CRR 4 # 4 - 11,CO,SANTANDER,AGUADA,3212087249,mariposaazul.77@outlook.es,CO-SAN-68013
3,ALCALDIA LA LLANADA,8001498940,"Colombia, Nariño , La Llanada",No aplica/No pertenece,CO1.BDOS.3667289,CO1.PCCNTR.4358427,MC 083 2022,Cerrado,SUMINISTRO DE ELEMENTOS DE PAPELERÍA Y MATERIA...,Suministros,...,3241046718,No Provisto,HADES-GROUP@HOTMAIL.COM,CARRERA 4 NO 03 - 36,CO,NARIÑO,SANDONa,3241046718,hades-group@hotmail.com,CO-NAR-52683
4,EMPRESA SOCIAL DEL ESTADO E.S.E. HOSPITAL REGI...,807008857,"Colombia, Norte de Santander , Tibú",Salud y Protección Social,CO1.BDOS.3680738,CO1.PCCNTR.4358085,088-2023,En ejecución,PRESTACION DE SERVICIOS EN APOYO A LA OFICINA ...,Decreto 092 de 2017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330326,ICBF REGIONAL QUINDIO,899999239,"Colombia, Antioquia , Armenia",Salud y Protección Social,CO1.BDOS.7685935,CO1.PCCNTR.7548611,63001512025,Activo,COMPLEMENTAR LA ATENCIÓN DE NIÑOS Y NIÑAS A TR...,Otro,...,3232248854,No Provisto,GERENCIA@COOHOBIENESTAR.ORG,CALLE 7 # 16-16 BARRIO GALAN,CO,QUINDiO,No Provisto,3208845369,ceroasiemprecoohobienestar@coohobienestar.org,CO-QUI
330327,ICBF REGIONAL QUINDIO,899999239,"Colombia, Antioquia , Armenia",Salud y Protección Social,CO1.BDOS.7685935,CO1.PCCNTR.7548611,63001512025,Activo,COMPLEMENTAR LA ATENCIÓN DE NIÑOS Y NIÑAS A TR...,Otro,...,3232248854,No Provisto,GERENCIA@COOHOBIENESTAR.ORG,CALLE 7 # 16-16 BARRIO GALAN,CO,QUINDiO,No Provisto,3208845369,ceroasiemprecoohobienestar@coohobienestar.org,CO-QUI
330328,ICBF REGIONAL QUINDIO,899999239,"Colombia, Antioquia , Armenia",Salud y Protección Social,CO1.BDOS.7685352,CO1.PCCNTR.7548317,63001492025,Activo,COMPLEMENTAR LA ATENCIÓN DE NIÑOS A TRAVÉS DE ...,Otro,...,7459341,7459341,ASOINFANCIA@GMAIL.COM,CALLE 12 NO. 21-26 BARRIO LA CABAÑA,CO,QUINDiO,ARMENIA,3117322864,asoinfancia@gmail.com,CO-QUI-63001
330329,ICBF REGIONAL QUINDIO,899999239,"Colombia, Antioquia , Armenia",Salud y Protección Social,CO1.BDOS.7686218,CO1.PCCNTR.7548711,63001522025,Activo,COMPLEMENTAR LA ATENCIÓN DE NIÑOS Y NIÑAS A TR...,Otro,...,3188894647,No Provisto,FESANCO98@GMAIL.COM,CARRERA 11 #10NORTE-55 EN LA CASTELLANA EN ARM...,CO,QUINDiO,ARMENIA,3188894647,fesanco98@gmail.com,CO-QUI-63001


In [57]:
# Definir las categorías y palabras clave
categorias = {
    "Obra": ["obra", "obras", "mantenimiento", "rehabilitación", "reparación", "adecuación", "carretera", "vial"],
    "Interventoría": ["interventoría", "supervisión", "intervenir"],
    "Topografía": ["topografía", "levantamiento topográfico", "planos"],
    "Drones": ["drone", "dron", "fotogrametría", "lidar", "geomática", "geográfica", "UAV"],
    "Agricultura": ["agricultura", "agrícola", "cultivo", "agro", "rural"],
    "Suministros": ["suministro", "suministros", "adquisición", "compra", "adquisición"],
    "Construcción": ["construcción", "infraestructura", "edificación"]
}

# Función para clasificar según palabras clave
def asignar_categoria(texto):
    if pd.isna(texto):  # Manejar valores nulos
        return "Otras categorías"

    texto = texto.lower()  # Convertir a minúsculas para evitar problemas de mayúsculas/minúsculas
    for categoria, palabras in categorias.items():
        if any(palabra in texto for palabra in palabras):
            return categoria
    return "Otras categorías"

# Aplicar la función a la nueva columna
df_final["Categoría"] = df_final["Objeto del Contrato"].apply(asignar_categoria)
df_final.head()

,Nombre Entidad,Nit Entidad,Localización,Sector,Proceso de Compra,ID Contrato,Referencia del Contrato,Estado Contrato,Descripcion del Proceso,Tipo de Contrato,...,fax,correo,direccion,pais,departamento,municipio,telefono_representante_legal,correo_representante_legal,ubicacion,Categoría
0,EMPRESA SOCIAL DEL ESTADO DEL MUNICIPIO DE VIL...,822002459,"Colombia, Meta , Villavicencio",Salud y Protección Social,CO1.BDOS.3677639,CO1.PCCNTR.4364202,039-2023,terminado,PRESTACION DE SERVICIOS PROFESIONALES COMO BAC...,Decreto 092 de 2017,...,No Provisto,DANAVHC1003@GMAIL.COM,CALLE 39 C N 7-22 MANANTIAL,CO,META,VILLAVICENCIO,3228982864,danavhc1003@gmail.com,CO-MET-50001,Otras categorías
1,CENTRO DE DIAGNOSTICO AUTOMOTOR DE NARIÑO LTDA,800206629,"Colombia, Nariño , Pasto",Transporte,CO1.BDOS.3692626,CO1.PCCNTR.4365365,PS2023-002,En ejecución,Prestar los servicios profesionales de Revisor...,Decreto 092 de 2017,...,7315832,HWILSONH11@GMAIL.COM,CALLE 19 NO. 28 - 72 OFICINA 302,CO,NARIÑO,PASTO,3187767039,codigosdiangmf@gmail.com,CO-NAR-52001,Otras categorías
2,RNEC,899999040,"Colombia, Bogotá, Bogotá",No aplica/No pertenece,CO1.BDOS.3691081,CO1.PCCNTR.4364799,DS-001-AGUADA-2023-RNEC-SANTANDER,Cerrado,El ARRENDADOR se compromete a dar en arrendami...,Arrendamiento de inmuebles,...,No Provisto,MARIPOSAAZUL.77@OUTLOOK.ES,CRR 4 # 4 - 11,CO,SANTANDER,AGUADA,3212087249,mariposaazul.77@outlook.es,CO-SAN-68013,Otras categorías
3,ALCALDIA LA LLANADA,8001498940,"Colombia, Nariño , La Llanada",No aplica/No pertenece,CO1.BDOS.3667289,CO1.PCCNTR.4358427,MC 083 2022,Cerrado,SUMINISTRO DE ELEMENTOS DE PAPELERÍA Y MATERIA...,Suministros,...,No Provisto,HADES-GROUP@HOTMAIL.COM,CARRERA 4 NO 03 - 36,CO,NARIÑO,SANDONa,3241046718,hades-group@hotmail.com,CO-NAR-52683,Suministros
4,EMPRESA SOCIAL DEL ESTADO E.S.E. HOSPITAL REGI...,807008857,"Colombia, Norte de Santander , Tibú",Salud y Protección Social,CO1.BDOS.3680738,CO1.PCCNTR.4358085,088-2023,En ejecución,PRESTACION DE SERVICIOS EN APOYO A LA OFICINA ...,Decreto 092 de 2017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Otras categorías


In [58]:
# Asegurar que la columna "Fecha de Firma" es de tipo datetime
df_final["Fecha de Firma"] = pd.to_datetime(df_final["Fecha de Firma"], errors='coerce')

# Filtrar solo las filas donde el año de "Fecha de Firma" sea 2023
df_2023 = df_final[df_final["Fecha de Firma"].dt.year == 2023]
df_2024 = df_final[df_final["Fecha de Firma"].dt.year == 2024]
df_2025 = df_final[df_final["Fecha de Firma"].dt.year == 2025]

# Ver cuántas filas quedaron después del filtro
print(f"✅ Filtrado completado: {df_2023.shape[0]} filas con fecha en 2023.")
print(f"✅ Filtrado completado: {df_2024.shape[0]} filas con fecha en 2024.")
print(f"✅ Filtrado completado: {df_2025.shape[0]} filas con fecha en 2025")


✅ Filtrado completado: 148412 filas con fecha en 2023.
✅ Filtrado completado: 163869 filas con fecha en 2024.
✅ Filtrado completado: 18050 filas con fecha en 2025
